In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing import image
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Define model architecture
def create_asl_image_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Path to train directory
train_dir = 'asl_alphabet_train\asl_alphabet_train'

: 

In [ ]:
# Data generators for training data
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(64, 64), batch_size=32, class_mode='sparse')

Found 87000 images belonging to 29 classes.


In [ ]:
# Create and train the model
input_shape = (64, 64, 3)
num_classes = len(os.listdir(train_dir))  # Number of folders in the train directory corresponds to the number of classes
asl_image_model = create_asl_image_model(input_shape, num_classes)

In [ ]:
checkpoint = ModelCheckpoint(
    'sign_language.model.h5',
    monitor='accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

Early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    patience=8,
    verbose=1,
    restore_best_weights=True
)


In [ ]:
# Train the model
asl_image_model.fit(train_generator, epochs=10, callbacks=[checkpoint,Early_stop])

Epoch 1/10
2719/2719 [==============================] - ETA: 0s - loss: 0.3233 - accuracy: 0.8904
Epoch 1: accuracy improved from -inf to 0.89039, saving model to /content/drive/MyDrive/predicting_sign_language/sign_language.model.h5
2719/2719 [==============================] - 181s 66ms/step - loss: 0.3233 - accuracy: 0.8904
Epoch 2/10
   1/2719 [..............................] - ETA: 4:10 - loss: 0.3449 - accuracy: 0.8750

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2719/2719 [==============================] - ETA: 0s - loss: 0.1951 - accuracy: 0.9343
Epoch 2: accuracy improved from 0.89039 to 0.93431, saving model to /content/drive/MyDrive/predicting_sign_language/sign_language.model.h5
2719/2719 [==============================] - 179s 66ms/step - loss: 0.1951 - accuracy: 0.9343
Epoch 3/10
2719/2719 [==============================] - ETA: 0s - loss: 0.1422 - accuracy: 0.9518
Epoch 3: accuracy improved from 0.93431 to 0.95178, saving model to /content/drive/MyDrive/predicting_sign_language/sign_language.model.h5
2719/2719 [==============================] - 183s 67ms/step - loss: 0.1422 - accuracy: 0.9518
Epoch 4/10
2719/2719 [==============================] - ETA: 0s - loss: 0.1153 - accuracy: 0.9612
Epoch 4: accuracy improved from 0.95178 to 0.96124, saving model to /content/drive/MyDrive/predicting_sign_language/sign_language.model.h5
2719/2719 [==============================] - 179s 66ms/step - loss: 0.1153 - accuracy: 0.9612
Epoch 5/10
2719/27

In [ ]:
# Path to test directory
test_dir = 'asl_alphabet_test\asl_alphabet_test'

In [ ]:
# Function to load and preprocess an image
def load_and_preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((64, 64))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

In [ ]:
# Create a dictionary to map class indices to class names
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}

In [ ]:
# Predict on test images
test_images = [f for f in os.listdir(test_dir) if f.endswith(('jpg', 'jpeg', 'png'))]
predictions = []

for img_name in test_images:
    img_path = os.path.join(test_dir, img_name)
    img = load_and_preprocess_image(img_path)
    prediction = asl_image_model.predict(img)
    predicted_class_index = np.argmax(prediction)
    predicted_class_label = class_labels[predicted_class_index]
    predictions.append((img_name, predicted_class_label))

1/1 [==============================] - 0s 24ms/step


In [ ]:
# Display predictions
for img_name, predicted_class_label in predictions:
    print(f"Image: {img_name}, Predicted Class: {predicted_class_label}")

Image: V_test.jpg, Predicted Class: V
Image: nothing_test.jpg, Predicted Class: nothing
Image: L_test.jpg, Predicted Class: L
Image: O_test.jpg, Predicted Class: O
Image: X_test.jpg, Predicted Class: X
Image: F_test.jpg, Predicted Class: F
Image: H_test.jpg, Predicted Class: H
Image: J_test.jpg, Predicted Class: J
Image: Y_test.jpg, Predicted Class: Y
Image: R_test.jpg, Predicted Class: R
Image: G_test.jpg, Predicted Class: G
Image: A_test.jpg, Predicted Class: A
Image: W_test.jpg, Predicted Class: W
Image: I_test.jpg, Predicted Class: I
Image: B_test.jpg, Predicted Class: B
Image: space_test.jpg, Predicted Class: space
Image: N_test.jpg, Predicted Class: N
Image: E_test.jpg, Predicted Class: E
Image: M_test.jpg, Predicted Class: M
Image: Z_test.jpg, Predicted Class: Z
Image: P_test.jpg, Predicted Class: P
Image: U_test.jpg, Predicted Class: U
Image: C_test.jpg, Predicted Class: C
Image: D_test.jpg, Predicted Class: D
Image: Q_test.jpg, Predicted Class: Q
Image: T_test.jpg, Predicted C